In [1]:
import tracking

from tracking import util
from tracking import layout
from tracking import busters

from tracking.busters import Actions, Directions
from tracking.pacman_utils import NotebookGraphics
from tracking.inference import InferenceModule

rules = tracking.busters.BustersGameRules()
%autosave 0


Autosave disabled


# Probabilistic Pacman

## Introduction

*Note*: Like assignment 3, this exercise is adapted from the Berkeley AI course. You can access the origninal API description and some other material on the following site: http://ai.berkeley.edu/tracking.html

In the Pacman version of Ghostbusters, the goal is to hunt down scared but invisible ghosts. Pacman is equipped with a sonar sensor that provide noisy readings of the Manhattan distance to each ghost. The game ends when Pacman has eaten all the ghosts.

Your primary task in this project is to implement inference to track the ghosts based on the information retrieved by this sensor. Fortunately, Bayes Nets provide us with powerful tools for making the most of the information we have. Throughout the rest of this project, you will implement algorithms for performing exact inference using Bayes' Nets.

The colored squares in the visualization indicate the probability of a ghost being in that square, given the noisy distance readings provided to Pacman. 
Each block is colored according to the **maximum (among the different ghosts, if there is more than one)** probability to see a ghost there, so the visualization of the probabilities doesn't mix colors.

When evaluating your code with the autograder, it will be helpful to have some understanding of what the autograder is doing. There are 2 types of tests in this project. For tests of class `DoubleInferenceAgentTest`, the probability distributions computed by your code will be compared with the staff's distributions. The second type of test is `GameScoreTest` (for the third exercise), in which your `BustersAgent` will actually select actions for Pacman.


## Sensor Model

In this question, you will implement the `observe` method in the `ExactInference` class to correctly update the agent's belief distribution over ghost positions given an observation from Pacman's sensors. A correct implementation should also handle one special case: when a ghost is eaten, you should place that ghost in its prison cell, as described in the comments of `observe`.

The sensor model of Pacman's sonar describes the probability of receiving a sensor reading, given the correct distance to the ghost: $P(noisyDistance \vert trueDistance)$. The model is defined as follows: The noisy distances that Pacman receives are always non-negative, and always within 7 of the true distance. The probability of a distance reading decreases exponentially with its difference from the true distance. This model is implemented in the `busters.getObservationDistribution(noisyDistance)` method.

Hints:

- You are implementing the online belief update for observing new evidence. Before any readings, Pacman believes the ghost could be anywhere: a uniform prior (as implemented in  `initializeUniformly`). After receiving each sensor reading, the `observe` function is called, which must update the belief at every position.
- **Before typing any code, draw the Bayes net of the model, and write down the equation of the inference problem you are trying to solve**. Note that, even though the marginal observation probability at this time step $t$, $P(Obs_t)$ will show up in the equations, there is no need to compute it, since it is a constant factor present in the occupation probabilities for all cells.
- Try printing `noisyDistance`, `emissionModel`, and `pacmanPosition` (inside the `observe` function) to get started.
- In the Pacman display, high posterior beliefs are represented by bright colors, while low beliefs are represented by dim colors. You should start with a large cloud of belief that shrinks over time as more evidence accumulates.
- Beliefs are stored as `util.Counter` objects (like dictionaries) in a field called `self.beliefs`, which you should update. You should not need to store any evidence. The only thing you need to store in `ExactInference` is `self.beliefs`.

In [10]:
class ExactInference(InferenceModule):
    """
    The exact dynamic inference module should use forward-algorithm
    updates to compute the exact belief function at each time step.
    """
    def getBeliefDistribution(self):
        return self.beliefs
    
    def initializeUniformly(self, gameState):
        """Begin with a uniform distribution over ghost positions.
        
        No need to implement anything here"""
        self.beliefs = util.Counter()
        for p in self.legalPositions: self.beliefs[p] = 1.0
        self.beliefs.normalize()

    def observe(self, observation, gameState):
        """
        Updates beliefs based on the distance observation and Pacman's position.

        The emissionModel below provides the probability of the noisyDistance for any true
        distance you supply.  That is, it stores P(noisyDistance | TrueDistance).

        self.legalPositions is a list of the possible ghost positions (you
        should only consider positions that are in self.legalPositions).

        A correct implementation needs to handle the following special case:
          *  When a ghost is captured by Pacman, all beliefs should be updated so
             that the ghost appears with probability 1 in its prison cell, position 
             self.getJailPosition()

             You can check if a ghost has been captured by Pacman by
             checking if the noisyDistance is None (a noisy distance
             of None will be returned if, and only if, the ghost is
             captured).

        """
        noisyDistance = observation
        print noisyDistance
        emissionModel = busters.getObservationDistribution(noisyDistance)
        pacmanPosition = gameState.getPacmanPosition()

        "*** YOUR CODE HERE ***"

        # Replace this code with a correct observation update
        # Be sure to handle the "jail" edge case where the ghost is eaten
        # and noisyDistance is None
        allPossible = util.Counter()
        if noisyDistance == None:
            allPossible = util.Counter()
            allPossible[self.getJailPosition()] = 1.0
        else:
            for location in self.legalPositions:
                distance = util.manhattanDistance(location, pacmanPosition)
                allPossible[location] = emissionModel[distance] * self.beliefs[location]
        "*** END YOUR CODE HERE ***"

        allPossible.normalize()
        self.beliefs = allPossible

This is how you can run a game of pacman on one of the test maps using you inference method: First, you need to load a maze layout

In [11]:
lay = layout.getLayout('q1/1-ExactObserve')  # load the layout of the map.

Other possible layouts are (these are also the ones used in the test cases):

    q1/1-ExactObserve.lay
    q1/2-ExactObserve.lay
    q1/3-ExactObserve.lay
    q1/4-ExactObserve.lay
    q2/1-ExactElapse.lay
    q2/2-ExactElapse.lay
    q2/3-ExactElapse.lay
    q2/4-ExactElapse.lay
    q3/1-ExactObserveElapse.lay (2 ghosts)
    q3/2-ExactObserveElapse.lay (4 ghosts)
    
Then, Pacman and the ghosts in the game need to be assigned an `Agent` to determine their actions.

**The maximum number of ghosts is determined by the layout. It is one unless otherwise noted in the list above. Ghost agents get a single parameter, which should be consecutive numbers starting with one, such as**

    ghosts = [SeededRandomGhostAgent(1), GoSouthAgent(2), GoSouthAgent(3)]
    
** for a layout with 3 or more ghosts.**

In [12]:
# set up a game. note that layout is already set previously.
from tracking.bustersAgents import BustersAgent
from tracking.trackingTestClasses import SeededRandomGhostAgent, GoSouthAgent

gameDisplay = NotebookGraphics()  # initialize the display of the playing field, needed here for the Pacman agent

# set an agent for the ghost. 
ghosts = [SeededRandomGhostAgent(1)]  # controls the behavior of a random ghost
# or
ghosts = [GoSouthAgent(1)]  # controls the behavior of a ghost that randomly tries to go south

mr_pacman = BustersAgent(ghostAgents=ghosts, display=gameDisplay, inferenceType=ExactInference, elapseTimeEnable=False)  # controls the pacman behavior


<IPython.core.display.Javascript object>

To run the inference on pacman, you should set up a game as described above on one of the `q1` layouts, and observe how the probabilities, as indicated by the shading of the grid cells, behave.

As you watch the test cases, be sure that you understand how the squares converge to their final coloring. In test cases where Pacman is boxed in (which is to say, he is unable to change his observation point), why does Pacman sometimes have trouble finding the exact location of the ghost?

The game is initialized and run as in the previous assignment. Since Pacman can not reach the ghost in most of these layouts, the game does not end automatically, and a maximum number of moves should be given:

In [13]:
game = rules.newGame(lay, mr_pacman, ghostAgents=ghosts, display=gameDisplay, maxMoves=50)  # instantiate a Game instance, see below
game.run()  # run the game, until Pacman catches a ghost.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

KeyboardInterrupt: 

## Movement Model

In the previous question you implemented belief updates for Pacman based on his observations. Fortunately, Pacman's observations are not his only source of knowledge about where a ghost may be. Pacman also has knowledge about the ways that a ghost may move; namely that the ghost can not move through a wall or more than one space in one timestep.

To understand why this is useful to Pacman, consider the following scenario in which there is Pacman and one Ghost. Pacman receives many observations which indicate the ghost is very near, but then one which indicates the ghost is very far. The reading indicating the ghost is very far is likely to be the result of a buggy sensor. Pacman's prior knowledge of how the ghost may move will decrease the impact of this reading since Pacman knows the ghost could not move so far in only one move.

In this question, you will implement the `elapseTime` method in `ExactInferenceWithElapse`. Your agent has access to the action distribution for any `GhostAgent`.
The movement model is the only source of information for Pacman in this exercise. **There is no need to incorporate observations here.**

For the tests in this question we will sometimes use a ghost with random movements (`SeededRandomGhostAgent`) and other times we will use the `GoSouthGhost`. This ghost tends to move south over time, and without any observations, Pacman's belief distribution should begin to focus around the bottom of the board. 

As you watch the visualization, remember that lighter squares indicate that pacman believes a ghost is more likely to occupy that location, and darker squares indicate a ghost is less likely to occupy that location. For which of the test cases do you notice differences emerging in the shading of the squares? Can you explain why some squares get lighter and some squares get darker?

We assume that ghosts still move independently of one another, so while you can develop all of your code for one ghost at a time, adding multiple ghosts should still work correctly.

In [ ]:
class ExactInferenceWithElapse(ExactInference):
    def elapseTime(self, gameState):
        """
        Updates self.beliefs according to a ghost's movement model in response to a 
        time step passing from the current state.
    
        The ghosts's movement model is not entirely stationary: it may depend on Pacman's
        current position (e.g., for DirectionalGhost).  However, this is not a problem,
        as Pacman's current position is known.
    
        In order to obtain the distribution over new positions for the
        ghost, given its previous position (oldPos) as well as Pacman's
        current position, use this line of code:
    
          newPosDist = self.getPositionDistribution(self.setGhostPosition(gameState, oldPos))
    
        newPosDist is a util.Counter object, where for each position p in self.legalPositions,
    
            newPostDist[p] = Pr( ghost is at position p at time t + 1 | ghost is at position oldPos at time t )
    
        (and also given Pacman's current position).
        
        You will need to compute multiple position distributions for a single update.
        """
    
        "*** YOUR CODE HERE ***"
        allPossible = util.Counter()
        for oldPos in self.legalPositions:
            newPosDist = self.getPositionDistribution(self.setGhostPosition(gameState, oldPos))
            print 'newPosDist', newPosDist
            for newPos, prob in newPosDist.items():
                allPossible[newPos] += prob*self.beliefs[oldPos]

        self.beliefs = allPossible
        print 'beliefs', self.beliefs

As before, a layout and agents are needed to play. This time, the Pacman agent gets the new inference method with the added movement model:

In [ ]:
lay = layout.getLayout('q2/3-ExactElapse')  # load the layout of the map.

from tracking.bustersAgents import BustersAgent
from tracking.trackingTestClasses import SeededRandomGhostAgent, GoSouthAgent

# set an agent for the ghost. 

# ghosts = [SeededRandomGhostAgent(1)]  # controls the behavior of a random ghost
# or
ghosts = [GoSouthAgent(1)]  # controls the behavior of a ghost that randomly tries to go south

gameDisplay = NotebookGraphics()  # initialize the display of the playing field
mr_pacman = BustersAgent(ghostAgents=ghosts, display=gameDisplay, inferenceType=ExactInferenceWithElapse, observeEnable=False)  # controls the pacman behavior

game = rules.newGame(lay, mr_pacman, ghostAgents=ghosts, display=gameDisplay, maxMoves=50)  # instantiate a Game instance, see below
game.run()  # run the game, until Pacman catches a ghost.


## Busters Agent

Now that Pacman knows how to use both his prior knowledge and his observations when figuring out where a ghost is, he is ready to hunt down ghosts on his own. This question will use your `observe` and `elapseTime` implementations together to compute a distribution over ghost positions.

Up to this point, Pacman has moved by randomly selecting a valid action.
Now, On top of the Bayesian model, you will implement a simple greedy hunting strategy. In this simple greedy strategy, Pacman assumes that each ghost is in its most likely position according to its beliefs, and always moves toward the closest ghost.

Implement the `chooseAction` method in `GreedyBustersAgent`. Your agent should first find the most likely position of each remaining (uncaptured) ghost, then choose an action that minimizes the distance to the closest ghost. If correctly implemented, your agent should win the game on the `q3/3-gameScoreTest` layout with a score greater than 700 at least 8 out of 10 times. 

*Note:* the autograder will also check the correctness of your inference directly, but the outcome of games is a reasonable sanity check.


Hints:

- When correctly implemented, your agent will thrash around a bit in order to capture a ghost.
- Make sure to only consider the living ghosts, as described in the docstring (comment).

In [ ]:
from tracking.bustersAgents import BustersAgent
from tracking.distanceCalculator import Distancer

class GreedyBustersAgent(BustersAgent):
    "An agent that charges the closest ghost."
    
    def __init__(self, *args, **kwargs):
        """No need to change anything here"""
        super(GreedyBustersAgent, self).__init__(*args, **kwargs)

    def registerInitialState(self, gameState):
        "Pre-computes the distance between every two points."
        BustersAgent.registerInitialState(self, gameState)
        self.distancer = Distancer(gameState.data.layout, False)

    def chooseAction(self, gameState):
        """
        First computes the most likely position of each ghost that
        has not yet been captured, then chooses an action that brings
        Pacman closer to the closest ghost (in maze distance!).

        To find the maze distance between any two positions, use:
        self.distancer.getDistance(pos1, pos2)

        To find the successor position of a position after an action:
        successorPosition = Actions.getSuccessor(position, action)

        livingGhostPositionDistributions, defined below, is a list of
        util.Counter objects equal to the position belief distributions
        for each of the ghosts that are still alive.  It is defined based
        on (these are implementation details about which you need not be
        concerned):

          1) gameState.getLivingGhosts(), a list of booleans, one for each
             agent, indicating whether or not the agent is alive.  Note
             that pacman is always agent 0, so the ghosts are agents 1,
             onwards (just as before).""

          2) self.ghostBeliefs, the list of belief distributions for each
             of the ghosts (including ghosts that are not alive).  The
             indices into this list should be 1 less than indices into the
             gameState.getLivingGhosts() list.

        """
        pacmanPosition = gameState.getPacmanPosition()
        legal = [a for a in gameState.getLegalPacmanActions()]
        livingGhosts = gameState.getLivingGhosts()
        livingGhostPositionDistributions = [beliefs for i, beliefs
                                            in enumerate(self.ghostBeliefs)
                                            if livingGhosts[i+1]]
        "*** YOUR CODE HERE ***"
        localMax = []
        for belief in livingGhostPositionDistributions:
            localMax.append(belief.argMax())
        goalCoordinate, goalProbability = None, 0
        for index, coordinate in enumerate(localMax):
            if livingGhostPositionDistributions[index][coordinate] >= goalProbability:
                goalCoordinate, goalProbability = coordinate, livingGhostPositionDistributions[index][coordinate]

        temp = []
        for action in legal:
            nextLocation = Actions.getSuccessor(pacmanPosition, action)
            temp.append((self.distancer.getDistance(nextLocation, goalCoordinate), action))
        return min(temp)[1]


In [ ]:
ghosts = [SeededRandomGhostAgent(1), SeededRandomGhostAgent(2)]  # controls the behavior of two ghosts

gameDisplay = NotebookGraphics()  # initialize the display of the playin0g field
mr_pacman = GreedyBustersAgent(ghostAgents=ghosts, display=gameDisplay, inferenceType=ExactInference)  # controls the pacman behavior

lay = layout.getLayout('q3/2-ExactObserveElapse')  # load the layout of the map

game = rules.newGame(lay, mr_pacman, ghostAgents=ghosts, display=gameDisplay, maxMoves=50)  # instantiate a Game instance, see below
game.run()  # run the game, until Pacman is caught by a ghost or there is no food left
